<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Gmail - Create GitHub issue on specific email

**Tags:** #gmail #github #email #issue #create #python

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Description:** This notebook will show how to create a GitHub issue from a specific email using Gmail and Python. It is usefull for organizations that need to track emails and create issues from them.

**References:**
- [Gmail API](https://developers.google.com/gmail/api)
- [GitHub API](https://developer.github.com/v3/)

## Input

### Import libraries

In [ ]:
import googleapiclient.discovery
import google_auth_oauthlib.flow
import google.auth.transport.requests
import requests

### Setup Variables
- `gmail_service`: Gmail API service
- `github_token`: GitHub token to access the API
- `email_id`: ID of the email to create the issue from

In [ ]:
gmail_service = None
github_token = None
email_id = None

## Model

### Create GitHub issue

Long description of the function without break

In [ ]:
# Get email content
email_content = gmail_service.users().messages().get(userId="me", id=email_id).execute()
# Create issue
url = "https://api.github.com/repos/{owner}/{repo}/issues"
headers = {"Authorization": "token {}".format(github_token)}
data = {
    "title": email_content["payload"]["headers"]["Subject"],
    "body": email_content["snippet"],
}
response = requests.post(url, headers=headers, data=data)

## Output

### Display result

In [ ]:
print(response.json())